<a href="https://colab.research.google.com/github/bimawardhana/traveloka-data-analysis/blob/master/traveloka-data-challenge/Solution_BimaSatria_SuryaWardhana.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **TRAVELOKA CUSTOMERS ANALYSIS & INSIGHT**

**BIMA SATRIA SURYA WARDHANA**

*Data Analyst (DA) — Marketing Analytics*


---



This analysis is used as a part of [Traveloka Data Challenge](https://traveloka.tau-data.com/2019/10/23/2019-tdc/) — using real cases scenario of Traveloka customers. Data used has been altered to match the challenges' requirement; therefore, some insights might not reflect the real world situation. However, some methods used in this analysis might be useful for any real-world analysis that might be conducted in the future.

Analysis will be triggered by patterns and characteristics found in Traveloka customers data based on the sample data given — *kindly refer to Data Exploration section for the sample data details* — so analysis with starts with data exploration to find pattern and characteristics, formulating problem statement, and then do deeper analysis on the data to answer the problem statement.

# 1. DATA EXPLORATION


This section covers several processes of data understanding, including: 1) ***extracting data*** from the sources, 2) ***data preparation*** — removing empty and unnecessary values as well as data schema adjustment, and 3) ***exploratory data analysis (EDA)*** to find patterns or important characteristics to be analysed further.



### 1.1. Customer Life Cycle Data

#### 1.1.1. Data Extraction

In [0]:
from google.colab import auth
from google.cloud import bigquery

In [7]:
# BIG QUERY AUTHENTICATION — interactively using web browser
try:
  auth.authenticate_user()
  print("AUTHENTICATION SUCCEEDED")
except:
  print("AUTHENTICATION FAILED")

# CONNECT TO PROJECT TO BE USED
try:
  client = bigquery.Client(project='tvlk-data-mktscience-dev')
  print("PROJECT CONNECTION SUCCEEDED")
except:
  print("PROJECT CONNECTION FAILED")

AUTHENTICATION SUCCEEDED
PROJECT CONNECTION SUCCEEDED


**Data context #1:**
Data is an all time data snapshot — being snapshotted per month. Data must be filtered only from the latest snapshot month.

In [29]:
# SEARCHING THE LATEST SNAPSHOT MONTH — to be used for filtering on data extraction
latest_snapshot_month = client.query(
  '''
    SELECT 
      CAST(MAX(snapshot_month) AS STRING) AS date
    FROM 
      `tvlk-data-mktscience-dev.data_challenge.customer_lifecycle`
  ''').to_dataframe().date[0]

print("LATEST SNAPSHOT MONTH: %s" % latest_snapshot_month)

LATEST SNAPSHOT MONTH: 2015-12-01


In [0]:
# IMPORTING DATA — analysis to be done in the dataframe
tvlk_customers = client.query(
    """
      SELECT
        *
      FROM
        `tvlk-data-mktscience-dev.data_challenge.customer_lifecycle`
      WHERE
        snapshot_month = \'%s\'
    """ % (latest_snapshot_month)).to_dataframe()

#### 1.1.2. Data Preparation

In [40]:
# VIEW DATA
tvlk_customers.head(10)

,row_num,profile_id,last_ads_id,snapshot_month,churn_cut_off,first_issued_month,last_issued_month,lifecycle,bucket_gbv_usd_all,bucket_issued_1y,bucket_visit_1y,bucket_search_1y,bucket_gbv_1y,bucket_netrev_1y,bucket_coupon_1y,visit_count_all,search_count_all,issued_count_all,gbv_usd_all,net_revenue_usd_all,coupon_usage_usd_all,burnt_external_points_all,burnt_internal_points_all,earned_points_all,selling_points_all,country,is_registered,is_subscribed,issued_count,sum_gbv_usd,net_revenue_usd,search_count,visit_count,coupon_usage_usd,earned_points,burnt_internal_points,burnt_external_points,selling_points,is_customer,is_churn,product,product_breadth
0,147551,796929,None,2015-12-01,24,2013-07-01,2013-08-01,09. at risk,1,1,1,1,1,1,5,12.0,12.0,12.0,0.0,0.0,0.0,None,None,None,None,ID,False,0,0.0,0.0,0.0,0,0,0.0,None,None,None,None,True,True,all,1
1,462257,320465,None,2015-12-01,24,2013-06-01,2013-06-01,09. at risk,1,1,1,1,1,1,5,5.0,5.0,5.0,0.0,0.0,0.0,None,None,None,None,ID,False,0,0.0,0.0,0.0,0,0,0.0,None,None,None,None,True,True,all,1
2,1021708,359569,None,2015-12-01,24,2013-06-01,2013-08-01,09. at risk,1,1,1,1,1,1,5,5.0,5.0,5.0,0.0,0.0,0.0,None,None,None,None,ID,False,0,0.0,0.0,0.0,0,0,0.0,None,None,None,None,True,True,all,1
3,1368922,481681,None,2015-12-01,24,2013-06-01,2013-06-01,09. at risk,1,1,1,2,1,1,5,4.0,8.0,4.0,0.0,0.0,0.0,None,None,None,None,ID,False,1,0.0,0.0,0.0,2,1,0.0,None,None,None,None,True,True,all,1
4,124127,186377,None,2015-12-01,24,2013-05-01,2013-05-01,09. at risk,1,1,1,1,1,1,5,4.0,4.0,4.0,0.0,0.0,0.0,None,None,None,None,ID,False,0,0.0,0.0,0.0,0,0,0.0,None,None,None,None,True,True,all,1
5,254226,497377,None,2015-12-01,24,2013-06-01,2013-08-01,09. at risk,1,1,1,1,1,1,5,10.0,10.0,10.0,0.0,0.0,0.0,None,None,None,None,ID,False,0,0.0,0.0,0.0,0,0,0.0,None,None,None,None,True,True,all,1
6,271805,355465,None,2015-12-01,24,2013-06-01,2013-06-01,09. at risk,1,1,1,1,1,1,5,4.0,4.0,4.0,0.0,0.0,0.0,None,None,None,None,ID,False,0,0.0,0.0,0.0,0,0,0.0,None,None,None,None,True,True,all,1
7,824814,156321,None,2015-12-01,24,2013-05-01,2013-08-01,11. hibernating,1,1,1,1,1,1,5,3.0,3.0,3.0,0.0,0.0,0.0,None,None,None,None,ID,False,1,0.0,0.0,0.0,0,0,0.0,None,None,None,None,True,True,all,1
8,1181019,279241,None,2015-12-01,24,2013-08-01,2013-08-01,11. hibernating,1,1,1,1,1,1,5,3.0,3.0,3.0,0.0,0.0,0.0,None,None,None,None,ID,False,0,0.0,0.0,0.0,0,0,0.0,None,None,None,None,True,True,all,1
9,1221682,421881,None,2015-12-01,24,2013-06-01,2013-06-01,11. hibernating,1,1,1,1,1,1,5,3.0,3.0,3.0,0.0,0.0,0.0,None,None,None,None,ID,False,0,0.0,0.0,0.0,0,0,0.0,None,None,None,None,True,True,all,1


In [38]:
# SHOWING DATA TYPES OF THE DATA
tvlk_customers.dtypes

row_num                        int64
profile_id                     int64
last_ads_id                   object
snapshot_month                object
churn_cut_off                  int64
first_issued_month            object
last_issued_month             object
lifecycle                     object
bucket_gbv_usd_all             int64
bucket_issued_1y               int64
bucket_visit_1y                int64
bucket_search_1y               int64
bucket_gbv_1y                  int64
bucket_netrev_1y               int64
bucket_coupon_1y               int64
visit_count_all              float64
search_count_all             float64
issued_count_all             float64
gbv_usd_all                  float64
net_revenue_usd_all          float64
coupon_usage_usd_all         float64
burnt_external_points_all     object
burnt_internal_points_all     object
earned_points_all             object
selling_points_all            object
country                       object
is_registered                   bool
i

Many of the data types in the tvlk_customers data doesn't match the required ones. This might be because of: 

*   ***NaN values in the fields*** — causing the data types of numeric columns to be OBJECT instead. NaN values need to be filled with the appropriate values (for NUMERIC: 0, for DATE: 1970-01-01, for STRING: UNKNOWN)
*   ***different data format*** — some boolean columns are notated as bool (TRUE/FALSE), some others are notated as integer (1/0). Standardisation for this is required.

In [0]:
tvlk_customers["last_ads_id"].fillna("UNKNOWN", inplace = True)

In [53]:
tvlk_customers.dtypes

row_num                        int64
profile_id                     int64
last_ads_id                   object
snapshot_month                object
churn_cut_off                  int64
first_issued_month            object
last_issued_month             object
lifecycle                     object
bucket_gbv_usd_all             int64
bucket_issued_1y               int64
bucket_visit_1y                int64
bucket_search_1y               int64
bucket_gbv_1y                  int64
bucket_netrev_1y               int64
bucket_coupon_1y               int64
visit_count_all              float64
search_count_all             float64
issued_count_all             float64
gbv_usd_all                  float64
net_revenue_usd_all          float64
coupon_usage_usd_all         float64
burnt_external_points_all     object
burnt_internal_points_all     object
earned_points_all             object
selling_points_all            object
country                       object
is_registered                   bool
i

In [11]:
# Draft — query output immediately

%%bigquery --project tvlk-data-mktanalytics-dev
SELECT 
  snapshot_month,
  COUNT(DISTINCT profile_id) AS total_customer
FROM 
  `tvlk-data-mktscience-dev.data_challenge.customer_lifecycle`
GROUP BY 
  1
ORDER BY 
  1 DESC

,snapshot_month,total_customer
0,2015-12-01,167143
1,2015-11-01,157234
2,2015-10-01,147407
3,2015-09-01,137833
4,2015-08-01,128518
5,2015-07-01,117180
6,2015-06-01,101719
7,2015-05-01,86827
8,2015-04-01,77782
9,2015-03-01,70435
